<a href="https://www.kaggle.com/code/lanaalnimreen/fork-of-brain-tumor-cnn?scriptVersionId=118608478" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Source of Dataset : https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from tensorflow.keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,add,MaxPool2D,Conv2D,Flatten
import cv2
from tensorflow.keras.utils import img_to_array

In [ ]:
labels = ['glioma','notumor','meningioma','pituitary']

# **load data**

In [ ]:
train_data = []
train_labels = []

test_data = []
test_labels = []

In [ ]:
image_size = 150
for i in labels:
  train_path= os.path.join("/kaggle/input/brain-tumor-mri-dataset/Training",i)
  for j in tqdm(os.listdir(train_path)):
        img = cv2.imread(os.path.join(train_path,j))
        img = cv2.resize(img,(image_size,image_size))
        train_data.append(img)
        train_labels.append(i)

In [ ]:
image_size = 150
for i in labels:
  test_path= os.path.join("/kaggle/input/brain-tumor-mri-dataset/Testing",i)
  for j in tqdm(os.listdir(test_path)):
        img = cv2.imread(os.path.join(test_path,j))
        img = cv2.resize(img,(image_size,image_size))
        test_data.append(img)
        test_labels.append(i)

In [ ]:
k=0
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']

fig, ax = plt.subplots(1,4,figsize=(20,20))
fig.text(s='Sample Image From Each Label',size=18,fontweight='bold',
             fontname='monospace',color=colors_dark[1],y=0.62,x=0.4,alpha=0.8)
for i in labels:
    j=0
    while True :
        if train_labels[j]==i:
            ax[k].imshow(train_data[j])
            ax[k].set_title(train_labels[j])
            ax[k].axis('off')
            k+=1
            break
        j+=1

# **rescaling**

In [ ]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [ ]:
np.unique(train_labels)

In [ ]:
train_data[0].shape

In [ ]:
np.min(train_data), np.max(train_data)

In [ ]:
np.min(test_data), np.max(test_data)

In [ ]:
train_data=train_data/255.0
test_data=test_data/255.0

# **shufling**

In [ ]:
train_data, train_labels = shuffle(train_data, train_labels, random_state=0)
test_data, test_labels = shuffle(test_data, test_labels, random_state=0)

# **Class Distribution**

In [ ]:
df_train = pd.DataFrame(train_labels,columns=['label'])
df_train.head()

In [ ]:
import seaborn as sns
sns.countplot(df_train['label'])

In [ ]:
df_test = pd.DataFrame(test_labels,columns=['label'])
df_test.head()

In [ ]:
import seaborn as sns
sns.countplot(df_test['label'])

# **encoding**

In [ ]:
train_labels[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
train_labels = to_categorical(train_labels,4)

train_labels[0]

In [ ]:
test_labels[0]

In [ ]:
test_labels

In [ ]:
le = LabelEncoder()
test_labels = le.fit_transform(test_labels)
test_labels = to_categorical(test_labels,4)


In [ ]:
test_labels[0]

# **create train,validation and test**

In [ ]:

train_datagen = ImageDataGenerator(        
        fill_mode="nearest",
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.1
)
test_datagen = ImageDataGenerator()


train_ds = train_datagen.flow(
  train_data, train_labels,
  subset="training",
  shuffle=True,
  batch_size=20)
val_ds = train_datagen.flow(
  train_data, train_labels,
  subset="validation",
  shuffle=True,
  batch_size=20)
test_ds = test_datagen.flow(
  test_data, test_labels,
  shuffle=False,
  batch_size=20)

# **Built Model**

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
resnet_model = ResNet50(weights='imagenet', input_shape=(150,150,3), include_top=False)
# Set all layers to non-trainable
for layer in resnet_model.layers[:171]:
    layer.trainable = False

for layer in resnet_model.layers[171:]:
    layer.trainable = True

In [ ]:
model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax')) # num labels = 4
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger
from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input
checkpoint_path = "myModel/mycnn_kaggle.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)


callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                       verbose=1,
                                       save_best_only=True,
                                       monitor = 'val_loss',
                                       mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.3,
                                        patience=2,
                                        verbose=1,
                                        mode='auto',
                                        min_delta=0.0001),
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    CSVLogger("myModel/mycnn_kaggle.log", separator=',', append=False),
]

In [ ]:
history = model.fit(train_ds,
                               epochs=5, 
                               verbose=1, 
                               validation_data = val_ds,
                               callbacks=callbacks)

In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
cnn = Sequential(
    [
        layers.Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation="relu", name="Conv1"),
        layers.MaxPooling2D(pool_size=(2, 2), name="Max_Pool_1"),
        
        layers.Conv2D(32, (3, 3), activation="relu", name="Conv2"),
        layers.MaxPooling2D(pool_size=(2, 2), name="Max_Pool_2"),
        
        layers.Conv2D(64, (3, 3), activation="relu", name="Conv3"),
        layers.MaxPooling2D(pool_size=(2, 2), name="Max_Pool_3"),
        
        layers.Flatten(),
        layers.Dense(256, activation="relu", name="fc1"),
        layers.Dropout(0.5),
        layers.Dense(4, activation="softmax", name="fc2"),    
    ]
)
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
cnn.summary()

In [ ]:
checkpoint_path = "myModel/mycnn_kaggle.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)


callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                       verbose=1,
                                       save_best_only=True,
                                       monitor = 'val_loss',
                                       mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.3,
                                        patience=2,
                                        verbose=1,
                                        mode='auto',
                                        min_delta=0.0001),
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    CSVLogger("myModel/mycnn_kaggle.log", separator=',', append=False),
]

In [ ]:
history_cnn = cnn.fit(train_ds,epochs=25,verbose=1, validation_data = val_ds,callbacks=callbacks)

In [ ]:
test_loss, test_acc = cnn.evaluate(test_ds, batch_size=32, callbacks= callbacks)
print("test loss: %.2f, test acc: %.2f"%(test_loss, test_acc))


In [ ]:
fig = plt.figure()
plt.plot(history_cnn.history['acc'], color='teal', label='accuracy')
plt.plot(history_cnn.history['val_acc'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
test_data[0]

# **Testing**

In [ ]:
pred = cnn.predict(test_ds)
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(pred.argmax(axis=1),test_labels.argmax(axis=1))

In [ ]:
cm=confusion_matrix(pred.argmax(axis=1),test_labels.argmax(axis=1))

In [ ]:
import itertools
from itertools import product
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Reds):
    
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm,labels)